# Objectives

The objective of this notebook is to provide a solution to the questions for the P2 Coding Challenge. This would showcase coding skills that covers data collection and analysis techniques.

## Interacting with the API to request the data

In [1]:
import pandas as pd
import requests
import json
from io import StringIO

In [2]:
# API call datastore_search

action = 'https://data.gov.au/data/api/3/action/datastore_search?'
resource_id = 'resource_id=809c77d8-fd68-4a2c-806f-c63d64e69842'

url = action + resource_id

def collect_data_API_request(url,rows_retrieve):
    ''' the function is used to interact with the API and collect data.
    url: url for http request to API
    rows_retrieve parameter: number of rows requested (int) or All
    '''
    
    # setting if rows_retrieve = 'All', code will get the total from the 
    if rows_retrieve == 'All':
        JSONContent = requests.get(url).json() # retrieves data from API as a dictionary
        content = json.dumps(JSONContent, indent=4, sort_keys=True) # generates as a json formatted str
        df_content = pd.read_json(JSONContent) # StringIO creates a file like object to be read by pd.read_json
        max_content = df_content['result']['total'] # get total number of rows from 
        limit = '&limit='+ str(max_content)
        
        url = url + limit
        
        # re-retrieve using the maximum number of rows from gathered from the API call
        JSONContent = requests.get(url).json()
        content = json.dumps(JSONContent, indent=4, sort_keys=True)
        df_content = pd.read_json(StringIO(content))
        df = pd.json_normalize(df_content['result']['records'])
    
    # setting if rows_retrieve = integer (will use &limit= number of rows requested)
    else:
        limit = '&limit='+ str(rows_retrieve)
        url = url + limit
        JSONContent = requests.get(url).json()
        content = json.dumps(JSONContent, indent=4, sort_keys=True)
        df_content = pd.read_json(StringIO(content))
        df = pd.json_normalize(df_content['result']['records'])
        
    return df

df = collect_data_API_request(url,rows_retrieve='All')

ValueError: Invalid file path or buffer object type: <class 'dict'>

# Data Exploration

The exploratory analysis aims to get acquainted with the data and answer the 3 questions given.

In [ ]:
# look into the top data example
df.head(3)

In [ ]:
# looking into data types post read
# Note: there are some null values which needs to be assessed
df.info()

In [ ]:
# show how many null values
df.isnull().sum()

In [ ]:
# show number of unique entries
df.nunique()

Investigating null values for Passengers

In [ ]:
# There are no zero values for the Passengers Out/In
print("Passengers_Out:" + str(df[df['Passengers_Out']==0].shape[0]))
print("Passengers_In:" + str(df[df['Passengers_In']==0].shape[0]))

In [ ]:
# null values then to be Freight/Mail flights, assume Passengers = 0 when values are null
df[df['Passengers_Out'].isnull()].head() 

In [ ]:
# example of a Non Passenger flight
df[df['Airline']=='FederalExpressCorporation']

Investigating null values for Freight

In [ ]:
# when Freight In is None there is value in Freight Out and vice versa
df[df['Freight_In_(tonnes)'].isnull()].head()

In [ ]:
# vise versa
df[df['Freight_Out_(tonnes)'].isnull()].head()

Investigating Mail out/in null values

In [3]:
# Similarly, shows Freight Airlines for Mail/Freight Out having None  when Freight/Main In have values
df[df['Mail_Out_(tonnes)'].isnull()].head()

NameError: name 'df' is not defined

In [4]:
# vise versa of previous
df[df['Mail_In_(tonnes)'].isnull()].head()

NameError: name 'df' is not defined

### Null values analysis and assumptions made
The analysis can assume that all "None or Null" values indicate a volume equivalent to 0. For example:  <br>

1. Airline with value "FederalExpressCorporation" (also known as FEDEX) which is a freight company and not a passenger company, explains the null values on Passengers_In/Out fields.

2. Freight/Mail "In" with null values will have Freight/Mail "Out" values, which could indicate that it is a specialised flight.

In [5]:
# Assign correct data types for fields.
df['Month'] = pd.to_datetime(df['Month'],format='%b-%y')

# assume null values as zero based on the previous analysis
df['Freight_In_(tonnes)'] = df['Freight_In_(tonnes)'].fillna('0').astype('float')
df['Freight_Out_(tonnes)'] = df['Freight_Out_(tonnes)'].fillna('0').astype('float')
df['Passengers_In'] = df['Passengers_In'].fillna('0').astype('float')
df['Passengers_Out'] = df['Passengers_Out'].fillna('0').astype('float')
df['Mail_In_(tonnes)'] = df['Mail_In_(tonnes)'].fillna('0').astype('float')
df['Mail_Out_(tonnes)'] = df['Mail_Out_(tonnes)'].fillna('0').astype('float')

df['Month_num'] = df['Month_num'].astype('int')

NameError: name 'df' is not defined

In [6]:
# basic summary statistics to look into volumes
df.describe()

NameError: name 'df' is not defined

# Questions and Answers

## How many passengers flew into and out of Australia for each airline in each month of 2019?

In [7]:
def passenger_total(df, year, Airline='All'):
    ''' This function calculates the total passengers flew into and out of australia
    df: dataframe
    year: required year
    '''
    # add the Passengers In and Out to calculate total passengers
    df['Passengers_In_Out'] = df.apply(lambda x: x['Passengers_In'] + x['Passengers_Out'],axis=1)
    
    # filter for specific year only (2019)
    df_year = df[df['Year']==year]
    
    # Answer: Dataframe below shows a sum of Monthly Passengers_In_Out per Airline on by specified year
    df_year = df_year.groupby(['Month_num','Airline']).Passengers_In_Out.sum().reset_index()
    
    if Airline != 'All':
        df_year = df_year[df_year['Airline']==Airline]
    return df_year

In [8]:
passenger_total(df, '2019')

NameError: name 'df' is not defined

In [9]:
# example look into 1 Airline
passenger_total(df, '2019',Airline='Virgin Australia')

NameError: name 'df' is not defined

## In the last 6 months which port country was responsible for the most passengers coming into the country?

In [10]:
# look into date range of the dataframe
print('minimum Month/Year: ' + str(df['Month'].min()))
print('maximum Month/Year: ' + str(df['Month'].max()))

NameError: name 'df' is not defined

In [11]:
# last 6 months from and to dates
df_last6months = df[(df['Month']>=pd.to_datetime('2021-01-01')) & (df['Month']<=pd.to_datetime('2021-06-01'))]
df_last6months = df_last6months.groupby(['Port_Country'])['Passengers_In'].sum().sort_values(ascending=True)
df_last6months.tail(10).plot(kind='barh',title='Top 10 Port Country with most passengers in the last 6 months');
# Answer = New Zealand

NameError: name 'df' is not defined

## What is the 3 month rolling average of freight arriving to Australia for each airline throughout 2018?

In [12]:
# select all months in 2018 & include 2 months of 2017 to complete the 3month rolling for January-18
df_2018 = df[(df['Month']>=pd.to_datetime('2017-11-01')) & (df['Month']<=pd.to_datetime('2018-12-01'))]
# calculate 3 months rolling average for each airline
df_2018_rolling3 = df_2018.groupby(['Month','Airline']).sum()['Freight_In_(tonnes)'].rolling(3).mean().reset_index()
df_2018_rolling3 = df_2018_rolling3[(df_2018_rolling3['Month']>=pd.to_datetime('2018-01-01')) & (df_2018_rolling3['Month']<=pd.to_datetime('2018-12-01'))]
# rename column
df_2018_rolling3 = df_2018_rolling3.rename(columns={'Freight_In_(tonnes)':'Freight_In_(tonnes)_3mths_rolling_Avg'})

NameError: name 'df' is not defined

In [13]:
# Answer: dataframe containing monthly rolling average per each arriving airline to Australia
df_2018_rolling3

NameError: name 'df_2018_rolling3' is not defined

In [14]:
# example country
df_2018_rolling3[df_2018_rolling3['Airline']=='Air Canada']

NameError: name 'df_2018_rolling3' is not defined

In [15]:
# Additional: 
# look into the mean of 3month rolling by Air Canada
df_2018_rolling3[df_2018_rolling3['Airline']=='Air Canada'].groupby('Airline').mean()

NameError: name 'df_2018_rolling3' is not defined

In [16]:
# mean of 3months rolling mean by Airline for 2018 
df_2018_rolling3.groupby('Airline').mean().sort_values(by='Freight_In_(tonnes)_3mths_rolling_Avg', ascending=False)

NameError: name 'df_2018_rolling3' is not defined

In [17]:
# END OF SOLUTION